In [9]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import signal
import pandas as pd
import time
from datetime import datetime
import WebTools
Tools = WebTools.Tools()
import DataframeShortcuts
DS = DataframeShortcuts.Shortcuts()
from tkinter import *
from tkinter import ttk
from PIL import ImageTk,Image
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from zipfile import ZipFile
import glob

# Creating timeout error to limit the runtime of a function if needed
class TimeoutException(Exception): # Creating custom error
    pass

def timeout_handler(): # Creating function to handle error
    print('Reached maximum allotted time')
    raise TimeoutException
    
signal.signal(signal.SIGALRM, timeout_handler);

In [15]:
def on_start():
   global run
   run = True

def on_stop():
   global run
   run = False

def download_url(url, save_path, chunk_size=1024, type='csv'):
   '''
   Save path is just the folder you want to download it in.

   Type must be a string, with the type of file you're downloading

   returns name of new file and its filepath. If downloaded is a zip, it will extract it and then 
   return the path to the folder along with the name of the folder
   '''
   files_in_directory = len(next(os.walk(save_path), (None, None, []))[2])
   name = save_path.split(sep='/')[-1] + '-' +str(files_in_directory) + '.' + type
   filepath = save_path + '/' + name + '/'

   r = requests.get(url, stream=True)
   with open(save_path + '/' + name, 'wb') as fd:
      for chunk in r.iter_content(chunk_size=chunk_size):
         fd.write(chunk)
   
   if type == 'zip':
      with ZipFile(save_path + '/' + name, 'r') as zObject:  
         zObject.extractall(path=save_path + '/')
         zObject.close()
      os.unlink(filepath)
      filepath = max(glob.glob(os.path.join(save_path, '*/')), key=os.path.getmtime)
      name = filepath.split(sep='/')[-2]

   return name,filepath[0:-1]

def autoprocess(data_path,dl_folder):
   try:
      data = pd.read_csv(data_path,low_memory=False)
   except:
      try:
         filename = data_path.split(sep='/')[-1]
         data_folder = dl_folder + '/' + filename.split(sep='.')[0]
         if not os.path.isdir(data_folder):
            os.mkdir(data_folder)
         failed_to_process = open(data_folder + '/failed_to_process.txt','a')
         e = datetime.now()
         failed_to_process.write(f'{filename} failed to open on {str(e.year)}, {str(e.month)}, {str(e.day)}\n')
         failed_to_process.close()
         return
      except:
         return

   failed1 = [False for idx,col in enumerate(data) if data.dtypes[idx] != 'object']
   filename = data_path.split(sep='/')[-1].split(sep='.')[0]
   data_folder = dl_folder + '/' + filename.split(sep='.')[0] + '_Data'
   if not os.path.isdir(data_folder):
      os.mkdir(data_folder)
      os.mkdir(data_folder + '/Histograms')
      os.mkdir(data_folder + '/Plots')

   for idx,col in enumerate(data):
      if data.dtypes[idx] != 'object':
         signal.alarm(5)
         try:            
            fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(8,8))
            ax.hist(np.array(data[col]))
            ax.set_xlabel(col)
            ax.set_title('Autogenerated Histogram' + ' ' + str(idx + 1))
            ax.set_facecolor('#ADD8E6')
            ax.set_axisbelow(True)
            ax.yaxis.grid(color='white', linestyle='-')
            fig.savefig(data_folder + '/Histograms/' + filename + '_' + 'Histogram-' + str(idx + 1) + '.png')
            plt.close('all')
         except:
            plt.close('all')
            failed1[idx] = True
            pass
         else:
            signal.alarm(0)
   
   failed2 = [False for idx,col in enumerate(data) if data.dtypes[idx] != 'object']
   for idx,col in enumerate(data):
      if data.dtypes[idx] != 'object':
         signal.alarm(5)
         try:            
            fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(8,8))
            ax.plot([idx for idx,j in enumerate(data[col])],np.array(data[col]))
            ax.set_xlabel(col)
            ax.set_title('Autogenerated Plot' + ' ' + str(idx + 1))
            ax.set_facecolor('#ADD8E6')
            ax.set_axisbelow(True)
            ax.yaxis.grid(color='white', linestyle='-')
            ax.xaxis.grid(color='white', linestyle='-')
            fig.savefig(data_folder + '/Plots/' + filename + '_' + 'Plot-' + str(idx + 1) + '.png')
            plt.close('all')
         except:
            plt.close('all')
            failed2[idx] = True
            pass
         else:
            signal.alarm(0)
   
   if all(failed1) and all(failed2):
      filename = data_path.split(sep='/')[-1]
      failed_to_process = open(data_folder + '/failed_to_process.txt','a')
      e = datetime.now()
      failed_to_process.write(f'{filename} failed to process both histograms and plots on {str(e.year)}, {str(e.month)}, {str(e.day)}\n')
      failed_to_process.close()
      return
   elif all(failed1) and not all(failed2):
      filename = data_path.split(sep='/')[-1]
      failed_to_process = open(data_folder + '/failed_to_process.txt','a')
      e = datetime.now()
      failed_to_process.write(f'{filename} failed to process histograms on {str(e.year)}, {str(e.month)}, {str(e.day)}\n')
      failed_to_process.close()
      return
   elif all(failed2) and not all(failed1):
      filename = data_path.split(sep='/')[-1]
      failed_to_process = open(data_folder + '/failed_to_process.txt','a')
      e = datetime.now()
      failed_to_process.write(f'{filename} failed to process plots on {str(e.year)}, {str(e.month)}, {str(e.day)}\n')
      failed_to_process.close()
      return
   return

def main():
   if run:
      # Check if data folder exists
      data_folder_path = sys.path[0] + '/Data'
      if not os.path.isdir(data_folder_path):
         print('Directory "Data" does not exist in current directory. Creating Directory.')
         os.mkdir(path=data_folder_path)

      # Check if websites.csv exists
      websites_csv_path = sys.path[0] + '/websites.csv'
      if not os.path.isfile(websites_csv_path):
         sys.exit('Necessary file "websites.csv" does not exist in current directory. Exiting Program.')

      # 1. Read in csv with websites
      try:
         df = pd.read_csv(websites_csv_path,header=0)
         df = df.reset_index(drop=True)
      except:
         sys.exit('Failed to open websites.csv. Exiting Program.')

      for idx,info in df.iterrows():

         # 2: Iterate over all entries to check if enough time has passed

         if ((round(time.time(),0) - info[3]) >= 100):
            df.iloc[idx] = [info[0],info[1],info[2],int(time.time())]
            
            dl_folder = data_folder_path + '/' + info[0]
            if os.path.isdir(dl_folder):
               try:
                  filename,filepath = download_url(info[1],dl_folder,type=info[2])
                  autoprocess(filepath,dl_folder)
               except:
                  pass
            else:
               try:
                  os.mkdir(dl_folder)
                  filename,filepath = download_url(info[1],dl_folder,type=info[2])
                  autoprocess(filepath,dl_folder)
               except:
                  pass      
      
      # 3. Check to see if user asked for entry to be deleted <- Not sure if this will get implemented

      # 4. Overwrite file 
      df.to_csv(sys.path[0] + '/websites.csv',index=False)
      print('looped')
      
   window.after(1, main)

In [16]:
window = Tk()
window.title('Transportation Data Manager')
window.iconbitmap(sys.path[0] + '/Resources/car2.ico')

frame = Frame(window)
frame.pack()

canvas = Canvas(frame, width=400, height=300, bg='#D3D3D3')
canvas.pack()

global run
run = False

start_label = Text(canvas,wrap=WORD,width=30,height=2,padx=6,pady=5,highlightthickness=0)
start_label.tag_configure('center',justify='center')  
start_label.insert('1.0','When pressed, this button will start the loop')
start_label.tag_add('center',1.0,'end')
start_label.place(relx = 0.3, rely = 0.4,anchor=CENTER)

start_button = Button(canvas, text="Start", command=on_start, padx=6,pady=5,highlightthickness=0)
start_button.place(relx=0.75, rely=0.4, anchor=CENTER)

end_label = Text(canvas,wrap=WORD,width=30,height=2,padx=6,pady=5,highlightthickness=0)
end_label.tag_configure('center',justify='center')  
end_label.insert('1.0','When pressed, this button will end the loop')
end_label.tag_add('center',1.0,'end')
end_label.place(relx = 0.3, rely = 0.6,anchor=CENTER)

end_button = Button(canvas, text="Stop", command=on_stop,padx=6,pady=5,highlightthickness=0)
end_button.place(relx=0.75,rely=0.6,anchor=CENTER)

info_label = Text(canvas,wrap=WORD,width=30,height=5,padx=6,pady=5,highlightthickness=0)
info_label.tag_configure('center',justify='center')  
info_label.insert('1.0','''This rudimentary GUI controls the script. 
New buttons and features may be added later if I can make it work''')
info_label.tag_add('center',1.0,'end')
info_label.place(relx=0.5, rely = 0.15,anchor=CENTER)

resized_img = Image.open(sys.path[0] + '/Resources/UT_logo.png').resize((130,100),Image.LANCZOS);
img = ImageTk.PhotoImage(resized_img)
canvas.create_image(350,260,image=img)

who_made_this = Text(canvas,wrap=WORD,width=35,height=3,padx=6,pady=5,highlightthickness=0)
who_made_this.tag_configure('center',justify='center')  
who_made_this.insert('1.0','''This program was made by Collin Dobson for the UTORII SMaRT internship''')
who_made_this.tag_add('center',1.0,'end')
who_made_this.place(relx=0.35,rely = 0.85,anchor=CENTER)

window.after(1, main)

window.mainloop()

Directory "Data" does not exist in current directory. Creating Directory.


<__array_function__ internals>:180: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
<__array_function__ internals>:180: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.


4856434
looped


<__array_function__ internals>:180: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
<__array_function__ internals>:180: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.


4856434
